In [4]:
import xarray as xr
import pandas as pd
import numpy as np
from glob import glob
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=15, threads_per_worker=1)
client  = Client(cluster, asynchronous=True)
client

2023-07-04 09:25:23,009 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-wk2n6zvw', purging
2023-07-04 09:25:23,010 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-85qrl4s_', purging
2023-07-04 09:25:23,010 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-lwgeym3y', purging
2023-07-04 09:25:23,010 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-zrsqitwn', purging
2023-07-04 09:25:23,010 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-6h7nn115', purging
2023-07-04 09:25:23,010 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-x1fckrtm', purging
2023-07-04 09:25:23,010 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-cc1spwn1', purging

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 15
Total threads: 15,Total memory: 125.57 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33791,Workers: 15
Dashboard: http://127.0.0.1:8787/status,Total threads: 15
Started: Just now,Total memory: 125.57 GiB
Comm: tcp://127.0.0.1:39253,Total threads: 1
Dashboard: http://127.0.0.1:43171/status,Memory: 8.37 GiB
Nanny: tcp://127.0.0.1:32879,


In [5]:
# load and build coastal mask
mask = xr.open_dataset('data/ERA5_MASKS.nc').coastmask_south

---

### ERA5

In [6]:
# load ERA5 coastal winds and mask
def preprocess(ds, mask=mask):
    print(ds.time[0].values)
    ds = ds.sortby('latitude').sortby('longitude')
    ds = ds.sel(latitude=slice(-40,-5)).isel(longitude=slice(0,-3))
    ds = ds.resample({'time':'d'}).mean()
    ds = ds.where(mask).mean('longitude').load()
    ds['ws'] = np.hypot(ds.u10, ds.v10)
    return ds['ws']
coastwinds = xr.open_mfdataset('~/storage/ERA5/COQUIMBO_VIENTO/WIND_Y*.nc',
                               preprocess=preprocess)
coastwinds = coastwinds.sortby('time')

2000-01-01T00:00:00.000000000
2000-10-01T00:00:00.000000000
2000-11-01T00:00:00.000000000
2000-12-01T00:00:00.000000000
2000-02-01T00:00:00.000000000
2000-03-01T00:00:00.000000000
2000-04-01T00:00:00.000000000
2000-05-01T00:00:00.000000000
2000-06-01T00:00:00.000000000
2000-07-01T00:00:00.000000000
2000-08-01T00:00:00.000000000
2000-09-01T00:00:00.000000000
2001-01-01T00:00:00.000000000
2001-10-01T00:00:00.000000000
2001-11-01T00:00:00.000000000
2001-12-01T00:00:00.000000000
2001-02-01T00:00:00.000000000
2001-03-01T00:00:00.000000000
2001-04-01T00:00:00.000000000
2001-05-01T00:00:00.000000000
2001-06-01T00:00:00.000000000
2001-07-01T00:00:00.000000000
2001-08-01T00:00:00.000000000
2001-09-01T00:00:00.000000000
2002-01-01T00:00:00.000000000
2002-10-01T00:00:00.000000000
2002-11-01T00:00:00.000000000
2002-12-01T00:00:00.000000000
2002-02-01T00:00:00.000000000
2002-03-01T00:00:00.000000000
2002-04-01T00:00:00.000000000
2002-05-01T00:00:00.000000000
2002-06-01T00:00:00.000000000
2002-07-01

In [8]:
for yr in range(2000,2021+1):
    print(yr)
    coastwinds.sel(time=str(yr)).to_netcdf('data/COQUIMBO_VIENTO/HOVMOLLER/ERA5_COASTWINDS_Y{}.nc'.format(yr))

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


---

### ECMWF ATMOSPHERE REFORECASTS

In [62]:
dirpaths = sorted(glob('data/S2S/reforecasts/*'))[:-40]
hovmoller_tropical   = []
hovmoller_coastnorth = []
hovmoller_coastsouth = []
for directory in dirpaths:
    print(directory)
    paths = glob(directory+'/*_WINDS.nc')
    data  = []
    for p in paths:
        d = xr.open_dataset(p).squeeze()
        d = d.sortby('lat').sortby('lon').sel({'lon':slice(265,295),'lat':slice(-45,0)})
        d = d.rename({'time':'leadtime'})
        d.coords['inittime'] = d.leadtime[0].values
        d['leadtime'] = np.arange(len(d.leadtime))
        data.append(d[['10u','10v']])
    data     = xr.concat(data, 'inittime').sortby('inittime').sortby('leadtime')
    data.coords['lon'] = (data.lon + 180)%360-180
    data = data.sortby('lon')
    ilat,ilon = data.lat, data.lon
    data = data.interp(lon=mask.longitude.values, lat=mask.latitude.values, method='nearest')
    data = data.where(mask.values==1).mean('lon')
    data = data.interp(lat=ilat, method='nearest').sel(lat=slice(-40,-5))
    data['ws'] = np.hypot(data['10u'], data['10v'])
    hovmoller_coastsouth.append(data)
    
#     tropical = data.where(masks.tropicalmask).mean('lat').dropna('lon').sortby('lon')
#     tropical.coords['index'] = ('lon',np.arange(len(tropical.lon)))
#     # tropical = tropical.to_dataset(name=var[1]+'_tropical')
    
#     coast_north = data.where(masks.coastmask_north).mean('lon').dropna('lat').sortby('lat')
#     coast_north.coords['index'] = ('lat', 1+tropical.index[-1].item()+np.arange(len(coast_north.lat)))
#     # coast_north = coast_north.to_dataset(name=var[1]+'_coastnorth')
    
#     coast_south = data.where(masks.coastmask_south).mean('lon').dropna('lat').sortby('lat', ascending=False)
#     coast_south.coords['index'] = ('lat', 1+tropical.index[-1].item()+np.arange(len(coast_south.lat)))
#     # coast_south = coast_south.to_dataset(name=var[1]+'_coastsouth')
    
#     hovmoller_tropical.append(tropical.rename({'dslm':'zos', 'param18.4.10':'T300'}))
#     hovmoller_coastnorth.append(coast_north.rename({'dslm':'zos', 'param18.4.10':'T300'}))
#     hovmoller_coastsouth.append(coast_south.rename({'dslm':'zos', 'param18.4.10':'T300'}))
    
# print('joining')
# hovmoller_tropical   = xr.concat(hovmoller_tropical, 'inittime').sortby('inittime')
# hovmoller_coastnorth = xr.concat(hovmoller_coastnorth, 'inittime').sortby('inittime')
print('join')
hovmoller_coastsouth = xr.concat(hovmoller_coastsouth, 'inittime').sortby('inittime')

data/S2S/reforecasts/2020-01-06
data/S2S/reforecasts/2020-01-09
data/S2S/reforecasts/2020-01-13
data/S2S/reforecasts/2020-01-16
data/S2S/reforecasts/2020-01-20
data/S2S/reforecasts/2020-01-23
data/S2S/reforecasts/2020-01-27
data/S2S/reforecasts/2020-01-30
data/S2S/reforecasts/2020-02-03
data/S2S/reforecasts/2020-02-06
data/S2S/reforecasts/2020-02-10
data/S2S/reforecasts/2020-02-13
data/S2S/reforecasts/2020-02-17
data/S2S/reforecasts/2020-02-20
data/S2S/reforecasts/2020-02-24
data/S2S/reforecasts/2020-02-27
data/S2S/reforecasts/2020-03-02
data/S2S/reforecasts/2020-03-05
data/S2S/reforecasts/2020-03-09
data/S2S/reforecasts/2020-03-12
data/S2S/reforecasts/2020-03-16
data/S2S/reforecasts/2020-03-19
data/S2S/reforecasts/2020-03-23
data/S2S/reforecasts/2020-03-26
data/S2S/reforecasts/2020-03-30
data/S2S/reforecasts/2020-04-02
data/S2S/reforecasts/2020-04-06
data/S2S/reforecasts/2020-04-09
data/S2S/reforecasts/2020-04-13
data/S2S/reforecasts/2020-04-16
data/S2S/reforecasts/2020-04-20
data/S2S

In [64]:
hovmoller_coastsouth.drop('height')

<xarray.Dataset>
Dimensions:   (inittime: 6480, leadtime: 47, lat: 23)
Coordinates:
  * leadtime  (leadtime) int64 0 1 2 3 4 5 6 7 8 ... 38 39 40 41 42 43 44 45 46
  * inittime  (inittime) datetime64[ns] 2000-01-06 2000-01-09 ... 2022-02-09
  * lat       (lat) float32 -39.0 -37.5 -36.0 -34.5 ... -10.5 -9.0 -7.5 -6.0
Data variables:
    10u       (inittime, leadtime, lat) float32 3.197 2.103 ... -1.062 -1.602
    10v       (inittime, leadtime, lat) float32 4.46 6.088 9.851 ... 4.666 5.664
    ws        (inittime, leadtime, lat) float32 5.488 6.441 ... 4.785 5.886

In [65]:
hovmoller_coastsouth.to_netcdf('data/COQUIMBO_VIENTO/HOVMOLLER/reforecast_winds_coastsouth.nc')

In [66]:
client.shutdown()